In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to fetch and parse the data
def fetch_data(url):
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
       #     print(response)

        # Parse the HTML content
        
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup)
        
        # Example: Get the title of the page
        title = soup.title.string if soup.title else "No Title Found"
        print(title)
        
        # Example: Get the first paragraph or some other content
        paragraph = soup.find('p').get_text() if soup.find('p') else "No Paragraph Found"
        
        return {"url": url, "title": title, "paragraph": paragraph}
    except requests.exceptions.RequestException as e:
        return {"url": url, "error": str(e)}

# URLs to scrape
urls = [  "https://yourstory.com/search?q=HDFC", "https://backend.finshots.in/backend/search/?q=HDFC"]

# Fetch data from both URLs
for url in urls:
    data = fetch_data(url)
    print(f"Data from {data['url']}:")
    print(f"Title: {data['title']}")
    print(f"First Paragraph: {data['paragraph']}\n")

In [47]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

# Function to fetch and parse the data

def fetch_data():
    df=pd.DataFrame(columns=['title','post_url','published_date','article'])
    # URLs to scrape
    keywords=["HDFC","Tata Motars"]
    # Fetch data from both URLs
    data=[]

    for keyword in keywords:
        url = f"https://backend.finshots.in/backend/search/?q={keyword}"
        try:
            # Fetch the webpage content
            response = requests.get(url)
            response.raise_for_status()  # Check for request errors
            #print(response.text)

            # Parse the HTML content
            
            soup = BeautifulSoup(response.content, 'html.parser')
            soup=json.loads(response.content)
            matches=soup["matches"]
            #print(matches[0])

            for match in matches:
                # Example: Get the title of the page
                title = match['title']
                #print(title)
                
                # url = match['post_url']
                post_url=match['post_url']
                published_date=match['published_date']
                req = requests.get(url=post_url)
                soup = BeautifulSoup(req.content, 'lxml')
                # Example: Get the first paragraph or some other content
                paragraph = soup.find('div',{"class":"post-content"}).text.strip()#.get_text() 
                #paragraph=soup.text.strip()
                row=pd.DataFrame({ "title": [title],"post_url": [post_url],'published_date':[published_date], "article": [paragraph]})
                #print(row)
                df=df._append(row,ignore_index=True)
            data.append(df)
        except requests.exceptions.RequestException as e:
            return {"url": url, "error": str(e)}

    return data

data=fetch_data()

In [48]:
def clean_data(data):
    for i in range(0,len(data)):
        # Sorting by column "published_date"
        data[i]=data[i].sort_values(by=['published_date'], ascending=False)
        data[i]=data[i].iloc[0:5,:]
        #print(data[i])
    return data

clean_data(data)[0]

,title,post_url,published_date,article
6,HDFC is in trouble — Has the elephant forgotte...,https://finshots.in/markets/hdfc-is-in-trouble/,2024-01-19T18:38:03.109000+00:00,"In today's Finshots, we explain why HDFC Bank'..."
8,"45 years later, HDFC bids goodbye to Dalal Street",https://finshots.in/archive/45-years-later-hdf...,2023-07-14T01:30:00+00:00,"On 12th July at 3:30 pm, the stock ticker HDFC..."
5,"HDFC bids farewell to its ₹10,000 crore educat...",https://finshots.in/archive/hdfc-bids-farewell...,2023-06-22T01:30:00+00:00,"In today’s Finshots, we explain why internatio..."
7,3 investing lessons from HDFC’s Prashant Jain,https://finshots.in/markets/investing-lessons-...,2022-09-09T10:46:32+00:00,In today's Finshots we describe a few lessons ...
3,Weekly Wrapup - Understanding the HDFC Merger,https://finshots.in/archive/weekly-wrapup-unde...,2022-04-08T16:51:54.934000+00:00,"Before we get to today's story, a quick recap ..."


In [50]:
def generate_sentiment_score(data):
    import random
    def mock_sentiment_api(text):
        sentiment_score = random.uniform(0, 1)
        return sentiment_score 

    for i in range(0,len(data)):
        data[i]['sentiment_score'] = data[i].apply(mock_sentiment_api, axis=1)
    return data
generate_sentiment_score(data)[0]

,title,post_url,published_date,article,sentiment_score
6,HDFC is in trouble — Has the elephant forgotte...,https://finshots.in/markets/hdfc-is-in-trouble/,2024-01-19T18:38:03.109000+00:00,"In today's Finshots, we explain why HDFC Bank'...",0.159779
8,"45 years later, HDFC bids goodbye to Dalal Street",https://finshots.in/archive/45-years-later-hdf...,2023-07-14T01:30:00+00:00,"On 12th July at 3:30 pm, the stock ticker HDFC...",0.632368
5,"HDFC bids farewell to its ₹10,000 crore educat...",https://finshots.in/archive/hdfc-bids-farewell...,2023-06-22T01:30:00+00:00,"In today’s Finshots, we explain why internatio...",0.426931
7,3 investing lessons from HDFC’s Prashant Jain,https://finshots.in/markets/investing-lessons-...,2022-09-09T10:46:32+00:00,In today's Finshots we describe a few lessons ...,0.757981
3,Weekly Wrapup - Understanding the HDFC Merger,https://finshots.in/archive/weekly-wrapup-unde...,2022-04-08T16:51:54.934000+00:00,"Before we get to today's story, a quick recap ...",0.832144


In [54]:
keys=["dd","ee"]
b={"ddd":1,'dee':2}
for i in range(0,len(keys)):
    a=b[f"d{keys[i]}"]
    print(a)

1
2


In [ ]:
import requests
import json
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import hashlib




def fetch_articles():
    # Code to scrape articles from finshots.in for keywords HDFC and Tata Motors
    query = ["HDFC","Tata Motors"]

    article_df_final = pd.DataFrame(columns=['aid','source','type','title','url','publishDate','text','query'])
    for each_query in query:
        article_df = pd.DataFrame(columns=['aid','source','type','title','url','publishDate','text'])
        r = requests.get(url="https://backend.finshots.in/backend/search/?q="+each_query)
        out = json.loads(r.content)
        print(out)
        matches = out['matches']
        for match in matches:
            title = match['title']
            publishDate = datetime.datetime.strptime(match['published_date'].split('T')[0],"%Y-%m-%d").date().strftime('%Y-%m-%d')
            url = match['post_url']
            req = requests.get(url=url)
            #print(req.text)
            soup = BeautifulSoup(req.content, 'lxml')
            p_tags = soup.find_all('div',{"class":"post-content"})
            text = ""
            hash_obj = hashlib.sha256(url.encode('utf-8'))
            hex_hash = hash_obj.hexdigest()
            aid = str(hex_hash)
            for p in p_tags:
                text+=p.text
            article_df = pd.concat([article_df, pd.DataFrame([[aid,'FinShots','Article',title,url,publishDate,text,each_query]],columns=['aid','source','type','title','url','publishDate','text','query'])],ignore_index=True)
        # Taking first five records
        article_df.sort_values(by='publishDate', ascending=False, inplace=True)
        first_five_records = article_df.head(5)
        article_df_final = pd.concat([article_df_final,first_five_records],ignore_index=True)
    
    
    
    return article_df_final

fetch_articles().head()


In [ ]:
file = open("dags/scripts/movie_analysis/ml-100k/u.user", "r")

In [67]:
user_df = pd.DataFrame(columns=['user_id', 'age', 'gender', 'occupation', 'zip'])
file = open("D:/airflow/dags/ml-100k/u.user", "r")
for each_row in file:
    user_df = pd.concat([user_df, pd.DataFrame([each_row.split('|')],\
                                                   columns=['user_id', 'age', 'gender', 'occupation', 'zip'])],\
                              ignore_index=True)
user_df['zip'] = [each_zip.rstrip("\n") for each_zip in user_df['zip']]
user_df['age'] = [int(age) for age in user_df['age']]

# Reading ratings data
rating_df= pd.read_csv("D:/airflow/dags/ml-100k/u.data", delimiter="\t", names=['user_id', 'item_id', 'rating', 'timestamp'])

# Reading item data
item_columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action',
                    'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item_data = pd.DataFrame(columns=item_columns)
genres = ['unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama',
              'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
              'Western']
item_file = open("D:/airflow/dags/ml-100k/u.item", "r", encoding='iso-8859-1')
for x in item_file:
    item_data = pd.concat([item_data, pd.DataFrame([x.split('|')], columns=item_columns)], ignore_index=True)
item_data['Western'] = [x.rstrip("\n") for x in item_data['Western']]
item_data['movie_id'] = [int(x) for x in item_data['movie_id']]
for genre in genres:
    item_data[genre] = [int(x) for x in item_data[genre]]


In [58]:
# Grouping by 'occupation' and calculating the mean age
mean_age_by_occupation = user_df.groupby('occupation')['age'].mean().reset_index()

# Display the result
print(mean_age_by_occupation)


       occupation        age
0   administrator  38.746835
1          artist  31.392857
2          doctor  43.571429
3        educator  42.010526
4        engineer  36.388060
5   entertainment  29.222222
6       executive  38.718750
7      healthcare  41.562500
8       homemaker  32.571429
9          lawyer  36.750000
10      librarian  40.000000
11      marketing  37.615385
12           none  26.555556
13          other  34.523810
14     programmer  33.121212
15        retired  63.071429
16       salesman  35.666667
17      scientist  35.548387
18        student  22.081633
19     technician  33.148148
20         writer  36.311111


In [59]:
# Merge rating_df with item_data to get movie titles
merged_df = pd.merge(rating_df, item_data, left_on='item_id', right_on='movie_id')

# Group by movie and calculate count of ratings and mean rating
movie_ratings = merged_df.groupby(['movie_id', 'movie_title']).agg(
    rating_count=('rating', 'count'),
    avg_rating=('rating', 'mean')
).reset_index()

# Filter movies that have been rated at least 35 times
filtered_movies = movie_ratings[movie_ratings['rating_count'] >= 35]

# Sort by average rating in descending order and select top 20
top_20_movies = filtered_movies.sort_values(by='avg_rating', ascending=False).head(20)

# Display the top 20 movies
print(top_20_movies[['movie_title', 'avg_rating', 'rating_count']])


                                           movie_title  avg_rating  \
407                              Close Shave, A (1995)    4.491071   
317                            Schindler's List (1993)    4.466443   
168                         Wrong Trousers, The (1993)    4.466102   
482                                  Casablanca (1942)    4.456790   
113  Wallace & Gromit: The Best of Aardman Animatio...    4.447761   
63                    Shawshank Redemption, The (1994)    4.445230   
602                                 Rear Window (1954)    4.387560   
11                          Usual Suspects, The (1995)    4.385768   
49                                    Star Wars (1977)    4.358491   
177                                12 Angry Men (1957)    4.344000   
512                              Third Man, The (1949)    4.333333   
133                                Citizen Kane (1941)    4.292929   
962            Some Folks Call It a Sling Blade (1993)    4.292683   
426                 

In [61]:
# Ensure 'user_id' and 'item_id' are of the same type in all DataFrames
user_df['user_id'] = user_df['user_id'].astype(int)
rating_df['user_id'] = rating_df['user_id'].astype(int)
rating_df['item_id'] = rating_df['item_id'].astype(int)
item_data['movie_id'] = item_data['movie_id'].astype(int)

# 1. Define age groups in user_df
age_bins = [20, 25, 35, 45, 100]  # Age groups: 20-25, 25-35, 35-45, 45+
age_labels = ['20-25', '25-35', '35-45', '45+']
user_df['age_group'] = pd.cut(user_df['age'], bins=age_bins, labels=age_labels, right=False)

# 2. Merge user_df, rating_df, and item_data on 'user_id' and 'item_id' (movie_id)
merged_df = pd.merge(rating_df, user_df, on='user_id')
merged_df = pd.merge(merged_df, item_data, left_on='item_id', right_on='movie_id')

# 3. Group by occupation, age_group, and genres
# List of genre columns in item_data
genres = ['unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 
          'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 
          'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Melt the dataframe to count ratings for each genre
genre_ratings = merged_df.melt(id_vars=['occupation', 'age_group', 'rating'], 
                               value_vars=genres, var_name='genre', value_name='is_genre')

# Filter rows where 'is_genre' is 1 (indicating the movie belongs to that genre)
genre_ratings = genre_ratings[genre_ratings['is_genre'] == 1]

# 4. Aggregate ratings by occupation, age group, and genre, and sum the ratings
genre_rating_summary = genre_ratings.groupby(['occupation', 'age_group', 'genre']).agg(
    total_ratings=('rating', 'sum')
).reset_index()

# 5. For each occupation and age group, find the genre with the highest total rating
top_genres_by_occupation_age = genre_rating_summary.loc[
    genre_rating_summary.groupby(['occupation', 'age_group'])['total_ratings'].idxmax()
]

# Display the top genres for each occupation and age group
print(top_genres_by_occupation_age[['occupation', 'age_group', 'genre', 'total_ratings']])


         occupation age_group  genre  total_ratings
7     administrator     20-25  Drama            376
26    administrator     25-35  Drama           3512
45    administrator     35-45  Drama           3672
64    administrator       45+  Drama           4204
83           artist     20-25  Drama            568
...             ...       ...    ...            ...
1508     technician       45+  Drama            982
1527         writer     20-25  Drama            782
1546         writer     25-35  Drama           3019
1565         writer     35-45  Drama           2246
1584         writer       45+  Drama           1546

[84 rows x 4 columns]


C:\Users\ankit\AppData\Local\Temp\ipykernel_12088\1688052622.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  genre_rating_summary = genre_ratings.groupby(['occupation', 'age_group', 'genre']).agg(
C:\Users\ankit\AppData\Local\Temp\ipykernel_12088\1688052622.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  genre_rating_summary.groupby(['occupation', 'age_group'])['total_ratings'].idxmax()


In [71]:
from scipy.stats import pearsonr
import numpy as np

# 1. Create a User-Movie Ratings Matrix
# Pivot rating_df to create user-movie matrix
user_movie_matrix = rating_df.pivot(index='user_id', columns='item_id', values='rating')

# 2. Calculate Similarity Between Movies
# Create a movie-to-movie similarity dictionary
def calculate_similarity(movie_id, movie_ratings_matrix, threshold=50):
    similar_movies = []
    target_movie_ratings = movie_ratings_matrix[movie_id]
    
    for other_movie_id in movie_ratings_matrix.columns:
        if other_movie_id == movie_id:
            continue
            
        # Get common users who rated both movies
        common_users = target_movie_ratings.dropna().index.intersection(
            movie_ratings_matrix[other_movie_id].dropna().index)
        
        # Only consider movies with enough co-occurrence
        if len(common_users) >= 50:
            # Calculate Pearson correlation for similarity
            similarity, _ = pearsonr(
                movie_ratings_matrix[movie_id].loc[common_users],
                movie_ratings_matrix[other_movie_id].loc[common_users]
            )
            
            # If similarity is greater than or equal to the similarity threshold (0.95)
            if similarity >= 0.0:
                similar_movies.append((other_movie_id, similarity, len(common_users)))
    
    return similar_movies

# 3. Find top 10 similar movies for a given movie
def find_top_similar_movies(movie_id, movie_ratings_matrix):
    similar_movies = calculate_similarity(movie_id, movie_ratings_matrix)
    
    # Sort movies by similarity score and co-occurrence count
    similar_movies_sorted = sorted(similar_movies, key=lambda x: (-x[1], -x[2]))[:10]
    
    return similar_movies_sorted

# Example: Finding top 10 similar movies to a given movie (say movie_id = 1)
movie_id = 1  # Replace with the ID of the movie you're interested in
top_similar_movies = find_top_similar_movies(movie_id, user_movie_matrix)

# Display the results (movie_id, similarity score, co-occurrence count)
print("Top 10 similar movies:")
for movie, similarity, co_occurrence in top_similar_movies:
    print(f"Movie ID: {movie}, Similarity: {similarity:.4f}, Co-occurrence: {co_occurrence}")


Top 10 similar movies:
Movie ID: 928, Similarity: 0.5491, Co-occurrence: 82
Movie ID: 969, Similarity: 0.4631, Co-occurrence: 58
Movie ID: 926, Similarity: 0.4580, Co-occurrence: 86
Movie ID: 378, Similarity: 0.4563, Co-occurrence: 74
Movie ID: 326, Similarity: 0.4548, Co-occurrence: 62
Movie ID: 588, Similarity: 0.4430, Co-occurrence: 166
Movie ID: 627, Similarity: 0.4375, Co-occurrence: 63
Movie ID: 72, Similarity: 0.4329, Co-occurrence: 100
Movie ID: 959, Similarity: 0.4284, Co-occurrence: 50
Movie ID: 418, Similarity: 0.4284, Co-occurrence: 105
